In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import hashlib

In [64]:
# %pip install faker

In [65]:
df = pd.read_excel('data/arquivo_anonimizado.xlsx')
df.drop(['Unnamed: 0', 'reprovacoes'], axis=1, inplace=True)

In [66]:
faker = Faker('pt_BR')

quantidade_de_nomes = df.shape[0]
nomes_completos = [faker.name() for _ in range(quantidade_de_nomes)]

df['nome_completo'] = nomes_completos

In [67]:
df.head(5).T

,0,1,2,3,4
matriculaDRE,0,1,2,3,4
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Informática,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por abandono,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 062...,2000/1 - MAA123 Algebra para Informatica - 009...,2000/1 - MAA123 Algebra para Informatica - 040...,2000/1 - MAA123 Algebra para Informatica - 081...
crPorPeriodo,2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001...,2000/1 - 7.6\n2000/2 - 6.5\n2001/1 - 5.1\n2001...,2000/1 - 3.3\n2000/2 - 4.7\n2001/1 - 0.7,2000/1 - 6.3\n2000/2 - 5.1\n2001/1 - 2.8\n2001...,2000/1 - 8.3\n2000/2 - 7\n2001/1 - 7.2\n2001/2...
craPorPeriodo,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7\n2001/1 - 6.4\n2001/2...,2000/1 - 3.3\n2000/2 - 3.9\n2001/1 - 3.2,2000/1 - 6.3\n2000/2 - 5.7\n2001/1 - 4.7\n2001...,2000/1 - 8.4\n2000/2 - 7.7\n2001/1 - 7.6\n2001...


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2817 entries, 0 to 2816
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   matriculaDRE               2817 non-null   int64  
 1   periodoIngressoUFRJ        2817 non-null   object 
 2   cursoIngressoUFRJ          2817 non-null   object 
 3   cursoAtual                 2817 non-null   object 
 4   periodoIngressoCursoAtual  2817 non-null   object 
 5   situacaoMatriculaAtual     2817 non-null   object 
 6   sexo                       2817 non-null   object 
 7   disciplinasCursadas        2650 non-null   object 
 8   crPorPeriodo               2644 non-null   object 
 9   craPorPeriodo              2642 non-null   object 
 10  periodosIntegralizados     2817 non-null   int64  
 11  cargaHorariaAcumulada      2817 non-null   int64  
 12  CRA                        2654 non-null   float64
 13  periodosTrancados          948 non-null    objec

# Processamento

Removendo alunos que não possui disciplinas cursas, pois se não há disciplinas, não há metricas a serem analisadas.

In [69]:
df.drop(['CRA', 'periodosCRMenor3', 'periodosTrancados', 'periodosCancelados'], axis=1, inplace=True)
df.dropna(subset=['disciplinasCursadas'], inplace=True)

## CR por Periodo

In [70]:
df['crPorPeriodo'] = df['crPorPeriodo'].astype(str)
df['crPorPeriodo'] = df['crPorPeriodo'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])
df_exploded = df.explode('crPorPeriodo')
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...
crPorPeriodo,"(2000/1, 7.6)","(2000/2, 7.1)","(2001/1, 5.9)","(2001/2, 6.8)","(2002/1, 6.3)"
craPorPeriodo,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...


In [71]:
df_crPeriodo = pd.DataFrame(df_exploded['crPorPeriodo'].tolist(), index=df_exploded.index, columns=('periodo','cr'))
df_crPeriodo['matriculaDRE'] = df_exploded['matriculaDRE']
df_crPeriodo

,periodo,cr,matriculaDRE
0,2000/1,7.6,0
0,2000/2,7.1,0
0,2001/1,5.9,0
0,2001/2,6.8,0
0,2002/1,6.3,0
...,...,...,...
2775,2022/1,7.5,2775
2776,2022/1,7.8,2776
2777,2022/1,7.9,2777
2778,2022/1,6.7,2778


In [72]:
# df_fato = pd.concat([df_exploded, df_crPeriodo], axis=1)
# df_fato.drop('crPorPeriodo', axis=1, inplace=True)
# df_fato.sample(5).T

## CRA por Periodo

In [73]:
df['craPorPeriodo'] = df['craPorPeriodo'].astype(str)
df['craPorPeriodo'] = df['craPorPeriodo'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])
df_exploded = df.explode('craPorPeriodo')
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...
crPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ..."
craPorPeriodo,"(2000/1, 7.6)","(2000/2, 7.4)","(2001/1, 6.9)","(2001/2, 6.9)","(2002/1, 6.8)"


In [74]:
df_craPeriodo = pd.DataFrame(df_exploded['craPorPeriodo'].tolist(), index=df_exploded.index, columns=('periodo','cra'))
df_craPeriodo['matriculaDRE'] = df_exploded['matriculaDRE']
df_craPeriodo

,periodo,cra,matriculaDRE
0,2000/1,7.6,0
0,2000/2,7.4,0
0,2001/1,6.9,0
0,2001/2,6.9,0
0,2002/1,6.8,0
...,...,...,...
2775,2022/1,7.5,2775
2776,2022/1,7.8,2776
2777,2022/1,7.9,2777
2778,2022/1,6.7,2778


In [75]:
# df_fato.drop('craPorPeriodo', axis=1, inplace=True)
# df_fato = pd.merge(df_fato, df_craPeriodo, on=['periodo', 'matriculaDRE'], how='left')
# df_fato.head(5).T

In [76]:
def processar_disciplinas(dados):
    dados = dados.astype(str)
    return dados.apply(lambda x: [split_disciplina(item) for item in x.split('\n')])

def split_disciplina(disciplina):
    detalhes_disciplinas = disciplina.split(' - ')
    if len(detalhes_disciplinas) == 5:
        detalhes_disciplinas[1] = ' - '.join(detalhes_disciplinas[1:3])
        del detalhes_disciplinas[2]
    return tuple(detalhes_disciplinas)


## Disciplinas Cursadas

Nessa seção vamos separar a lista de disciplinas em varias colunas, uma para cada disciplina

In [77]:
df['disciplinasCursadas'] = df['disciplinasCursadas'].astype(str)
# df['disciplinasCursadas'] = df['disciplinasCursadas'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])
df['disciplinasCursadas'] = processar_disciplinas(df['disciplinasCursadas'])

In [78]:
df_exploded = df.explode('disciplinasCursadas')
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,"(2000/1, MAA123 Algebra para Informatica, 060,...","(2000/1, MAB111 Fund da Computação Digital, 08...","(2000/1, MAB120 Computacao para Informatica, 0...","(2000/1, MAE111 Cálculo Infinitesimal I, 056, ...","(2000/1, MAE115 Cálculo Vetorial e G Analitica..."
crPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ..."
craPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ..."


Agora vamos pegar as informações de cada desciplina e desmembrar em novas colunas

In [79]:
df_exploded['disciplinasCursadas'] = df_exploded['disciplinasCursadas'].apply(lambda x: (
    x[0].replace(' ', ''),
    x[1].split()[0].replace(' ', ''),
    ' '.join(x[1].split()[1:]),
    x[2].replace(' ', ''),
    x[3].replace(' ', '')
))

In [80]:
df_disciplina = pd.DataFrame(df_exploded['disciplinasCursadas'].tolist(), index=df_exploded.index, columns=('periodo','cod_Disciplina','disciplina','grau_disciplina','situacao_disciplina'))
df_disciplina['matriculaDRE'] = df_exploded['matriculaDRE']
df_disciplina

,periodo,cod_Disciplina,disciplina,grau_disciplina,situacao_disciplina,matriculaDRE
0,2000/1,MAA123,Algebra para Informatica,060,Aprovado,0
0,2000/1,MAB111,Fund da Computação Digital,089,Aprovado,0
0,2000/1,MAB120,Computacao para Informatica,094,Aprovado,0
0,2000/1,MAE111,Cálculo Infinitesimal I,056,Aprovado,0
0,2000/1,MAE115,Cálculo Vetorial e G Analitica,085,Aprovado,0
...,...,...,...,...,...,...
2779,2022/1,ICP111,Fund Computação Digital,067,Aprovado,2779
2779,2022/1,ICP112,Sistemas de Informação,082,Aprovado,2779
2779,2022/1,ICP120,Computação I (CC),075,Aprovado,2779
2779,2022/1,ICP136,Introd Pensamento Dedutivo,081,Aprovado,2779


In [81]:
# df_fato.drop('disciplinasCursadas',axis=1, inplace=True)
# df_fato = pd.merge(df_fato, df_disciplina, on=['periodo', 'matriculaDRE'], how='left')
# df_fato.head(5).T

In [82]:
df_merge = pd.merge(df_disciplina, df_craPeriodo, on=['periodo', 'matriculaDRE'], how='left')
df_merge = pd.merge(df_merge, df_crPeriodo, on=['periodo', 'matriculaDRE'], how='left')
df_merge

,periodo,cod_Disciplina,disciplina,grau_disciplina,situacao_disciplina,matriculaDRE,cra,cr
0,2000/1,MAA123,Algebra para Informatica,060,Aprovado,0,7.6,7.6
1,2000/1,MAB111,Fund da Computação Digital,089,Aprovado,0,7.6,7.6
2,2000/1,MAB120,Computacao para Informatica,094,Aprovado,0,7.6,7.6
3,2000/1,MAE111,Cálculo Infinitesimal I,056,Aprovado,0,7.6,7.6
4,2000/1,MAE115,Cálculo Vetorial e G Analitica,085,Aprovado,0,7.6,7.6
...,...,...,...,...,...,...,...,...
98634,2022/1,ICP111,Fund Computação Digital,067,Aprovado,2779,7.8,7.8
98635,2022/1,ICP112,Sistemas de Informação,082,Aprovado,2779,7.8,7.8
98636,2022/1,ICP120,Computação I (CC),075,Aprovado,2779,7.8,7.8
98637,2022/1,ICP136,Introd Pensamento Dedutivo,081,Aprovado,2779,7.8,7.8


In [83]:
df_fato = pd.merge(df_merge, df, on=['matriculaDRE'], how='left')
df_fato.drop(['disciplinasCursadas','crPorPeriodo', 'craPorPeriodo'], axis=1, inplace=True)
df_fato.sample(5).T

,71426,77193,35712,45561,55025
periodo,2016/2,2014/2,2006/2,2008/2,2011/1
cod_Disciplina,MAB102,MAB112,FIT111,FIT111,MABX04
disciplina,Trab Grand Vol Dados (Dc e Bd),Sistemas de Informação,Física I,Física I,Monitoria II
grau_disciplina,090,071,060,052,100
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
matriculaDRE,1731,1901,821,1039,1321
cra,7.8,4.9,7.6,5.6,7.9
cr,8.2,4.8,7.1,6,7.2
periodoIngressoUFRJ,2013/1,2014/2,2006/1,2008/1,2010/1
cursoIngressoUFRJ,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação


In [84]:
df_fato[['ano', 'semestre']] = df_fato['periodo'].str.split('/', expand=True)
df_fato.head(5).T

,0,1,2,3,4
periodo,2000/1,2000/1,2000/1,2000/1,2000/1
cod_Disciplina,MAA123,MAB111,MAB120,MAE111,MAE115
disciplina,Algebra para Informatica,Fund da Computação Digital,Computacao para Informatica,Cálculo Infinitesimal I,Cálculo Vetorial e G Analitica
grau_disciplina,060,089,094,056,085
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
matriculaDRE,0,0,0,0,0
cra,7.6,7.6,7.6,7.6,7.6
cr,7.6,7.6,7.6,7.6,7.6
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática


# Realizando Cagas das Tabelas

## Dimenssão Aluno

In [85]:
colunas_aluno = ['sk_d_aluno', 'nome_completo', 'matriculaDRE', 'sexo']

In [86]:
d_aluno = df_fato[colunas_aluno[1:]]

# d_aluno.groupby(['nome_completo', 'matriculaDRE', 'sexo']).first().reset_index()
d_aluno = d_aluno.drop_duplicates().reset_index(drop=True)
d_aluno

,nome_completo,matriculaDRE,sexo
0,Otávio Aragão,0,M
1,Esther Araújo,1,M
2,Ryan Vieira,2,M
3,Miguel Pinto,3,M
4,Yasmin Sales,4,M
...,...,...,...
2645,Lucas Gabriel Ferreira,2775,M
2646,Isadora da Paz,2776,M
2647,Maria Luiza da Paz,2777,M
2648,Ana Julia da Paz,2778,M


In [87]:
d_aluno['sk_d_aluno'] = d_aluno.apply(lambda x: hashlib.sha256(str(x['matriculaDRE']).encode()).hexdigest(), axis=1)
d_aluno = d_aluno.reindex(columns=colunas_aluno)
d_aluno.to_csv('data/d_aluno.csv', index=False)
d_aluno

,sk_d_aluno,nome_completo,matriculaDRE,sexo
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,Otávio Aragão,0,M
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,Esther Araújo,1,M
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,Ryan Vieira,2,M
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,Miguel Pinto,3,M
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,Yasmin Sales,4,M
...,...,...,...,...
2645,3da16474e33b330b71e5b562cfcf7939bde3c12a08b746...,Lucas Gabriel Ferreira,2775,M
2646,9b3cea3bd8789ba47cc969bff5914da7ebac8e0052e282...,Isadora da Paz,2776,M
2647,5a48eed290f62c93553855c36c964e1ef16603d23dcce3...,Maria Luiza da Paz,2777,M
2648,a2541e84e912f4f5dd32ce131abdb6b1fdc9cf315790f1...,Ana Julia da Paz,2778,M


## Dimenssão Disciplina

In [88]:
# def criar_dimenssao(nome_dimenssao, colunas_dimenssao, hash):
#     dimenssao = df_fato[colunas_dimenssao[1:]]
#     dimenssao = dimenssao.drop_duplicates().reset_index(drop=True)
#     dimenssao[f'sk_d_{nome_dimenssao}'] = d_disciplina.apply(lambda x: hashlib.sha256(str(x[hash]).encode()).hexdigest(), axis=1)
#     d_disciplina = d_disciplina.reindex(columns=colunas_aluno)
#     d_disciplina.to_csv(f'data/d_{nome_dimenssao}.csv', index=False)
#     d_disciplina

In [89]:
colunas_disciplina = ['sk_d_disciplina', 'cod_Disciplina', 'disciplina']

In [90]:
d_disciplina = df_fato[colunas_disciplina[1:]]
d_disciplina = d_disciplina.drop_duplicates().reset_index(drop=True)
d_disciplina

,cod_Disciplina,disciplina
0,MAA123,Algebra para Informatica
1,MAB111,Fund da Computação Digital
2,MAB120,Computacao para Informatica
3,MAE111,Cálculo Infinitesimal I
4,MAE115,Cálculo Vetorial e G Analitica
...,...,...
1589,NEP144,Tóp Esp Polít Públ Dir Hum VI
1590,NEP142,Tóp Esp Polít Públ Dir Hum IV
1591,MAE245,Álgebra Linear II - EQ
1592,ICP114,Computação I Ep


In [91]:
d_disciplina['sk_d_disciplina'] = d_disciplina.apply(lambda x: hashlib.sha256(str(x['cod_Disciplina'] + x['disciplina']).encode()).hexdigest(), axis=1)
d_disciplina = d_disciplina.reindex(columns=colunas_disciplina)
d_disciplina.to_csv('data/d_disciplina.csv', index=False)
d_disciplina

,sk_d_disciplina,cod_Disciplina,disciplina
0,26ae25e81ccc85aecdad201a410a09485d415acd4daf6e...,MAA123,Algebra para Informatica
1,4ba51a95ba6087306319254f45d5ee5ad335922069282f...,MAB111,Fund da Computação Digital
2,09e5060ee94ea2b9604e23295927e68d40d481a362d81e...,MAB120,Computacao para Informatica
3,63637e47a78400c87bb1b82da2fc5b6c00e78c0c77c779...,MAE111,Cálculo Infinitesimal I
4,6f904cf605935af92e93713c9eead620c9c92cd971dfc4...,MAE115,Cálculo Vetorial e G Analitica
...,...,...,...
1589,f711fb63fc524f5758c111314b49b44d86a21e48bb458e...,NEP144,Tóp Esp Polít Públ Dir Hum VI
1590,eac458f82acdd92fd7aa393c4453804d5516adbbaa6158...,NEP142,Tóp Esp Polít Públ Dir Hum IV
1591,574c8aa6dc1f57bf5ca26827a0871cbd70ef3416288591...,MAE245,Álgebra Linear II - EQ
1592,9d40b24dedf530a6991c38a516e1275f152e62264a656e...,ICP114,Computação I Ep


## Dimenssão Periodo

In [92]:
colunas_periodo = ['sk_d_periodo', 'ano', 'semestre']

In [93]:
d_periodo = df_fato[colunas_periodo[1:]]
d_periodo = d_periodo.drop_duplicates().reset_index(drop=True)
d_periodo

,ano,semestre
0,2000,1
1,2000,2
2,2001,1
3,2001,2
4,2002,1
...,...,...
67,2019,0
68,2021,0
69,2013,3
70,2014,3


In [94]:
d_periodo['sk_d_periodo'] = d_periodo.apply(lambda x: hashlib.sha256(str(x['ano'] + x['semestre']).encode()).hexdigest(), axis=1)
d_periodo = d_periodo.reindex(columns=colunas_periodo)
d_periodo.to_csv('data/d_periodo.csv', index=False)
d_periodo

,sk_d_periodo,ano,semestre
0,fc82267b45dcbf8db9e4ec10055373cfff41efcbc5b83a...,2000,1
1,29433eae6f7f1308d9799275f3a90a0afe1fef0e1818a7...,2000,2
2,e4b8f2da99fd9407c758cd306f846562424ec5c38d3202...,2001,1
3,b6167a59dbfdca3d716a326a015dd8a00f417590eb34e2...,2001,2
4,c291fe829d40dc63fc4fa578f6208f425064945d31ca82...,2002,1
...,...,...,...
67,8ad00ecd5eb3a6ca42f90cc776b009968ec5865f2d37b3...,2019,0
68,fa122eae077795ac4ff0f80bb75550a9a2c30a192d035c...,2021,0
69,fa103adf3e582291297b8330060eace6bc1a14f8737cf2...,2013,3
70,10071c57cf3b2135572815c0243a079fc18936a1259c3d...,2014,3


## Dimenssão Curso

In [95]:
colunas_curso = ['sk_d_curso', 'cursoIngressoUFRJ', 'cursoAtual']

In [96]:
d_curso = df_fato[colunas_curso[1:]]
d_curso = d_curso.drop_duplicates().reset_index(drop=True)
d_curso

,cursoIngressoUFRJ,cursoAtual
0,Bacharelado em Informática,Bacharelado em Ciência da Computação
1,Bacharelado em Informática,Bacharelado em Informática
2,Bacharelado em Informática,Licenciatura em Matemática
3,Bacharelado em Informática,Mat Aplicada - Computação Científica
4,Matemática,Bacharelado em Ciência da Computação
...,...,...
73,Licenciatura em Física,Bacharelado em Ciência da Computação
74,Bacharelado em Ciência da Computação,Licenciatura em Geografia
75,Ciências Matemáticas e da Terra,Ciência da Computação
76,Bacharelado em Ciência da Computação,Engenharia Eletrônica e de Computação


In [97]:
d_curso['sk_d_curso'] = d_curso.apply(lambda x: hashlib.sha256(str(x['cursoIngressoUFRJ'] + x['cursoAtual']).encode()).hexdigest(), axis=1)
d_curso = d_curso.reindex(columns=colunas_curso)
d_curso.to_csv('data/d_curso.csv', index=False)
d_curso

,sk_d_curso,cursoIngressoUFRJ,cursoAtual
0,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,Bacharelado em Informática,Bacharelado em Ciência da Computação
1,dbde55e27224a8c832119eba0c54cb25a32bf6f6417dd2...,Bacharelado em Informática,Bacharelado em Informática
2,cb369a9ae66bcd4659f91ed5d88c2a7674ca54645d641a...,Bacharelado em Informática,Licenciatura em Matemática
3,0367c21aa27e715610d936288c8fb811fc6dd378b050b5...,Bacharelado em Informática,Mat Aplicada - Computação Científica
4,5f20305be16f0143bc9a1bcc213bc104dd93a64e5ad031...,Matemática,Bacharelado em Ciência da Computação
...,...,...,...
73,b04e50e1e81b07e4bba26b99505e3a21fd9160bdd6c664...,Licenciatura em Física,Bacharelado em Ciência da Computação
74,2d37b7f8c6308826f3cd805e314628baf4dede1e70bb0a...,Bacharelado em Ciência da Computação,Licenciatura em Geografia
75,acbc30f06ec39601955703a6f4fb05791a031a8f7755d1...,Ciências Matemáticas e da Terra,Ciência da Computação
76,a54d2fed0213e2b06b6fb51bf134c8f5920420505116c3...,Bacharelado em Ciência da Computação,Engenharia Eletrônica e de Computação


## Dimenssão Situação Academica

In [98]:
colunas_situacao_academica = ['sk_d_situacao_academica', 'situacaoMatriculaAtual']

In [99]:
d_situacao_academica = df_fato[colunas_situacao_academica[1:]]
d_situacao_academica = d_situacao_academica.drop_duplicates().reset_index(drop=True)
d_situacao_academica

,situacaoMatriculaAtual
0,Cancelada por conclusão de curso
1,Cancelada por abandono
2,Cancelada por transferência
3,Cancelada por abandono Definitivo
4,Cancelamento a pedido
5,Cancelamento por ultrapassagem do prazo máximo...
6,Cancelamento por opção de curso
7,Cancelada por rendimento escolar insuficiente
8,Cancelamento por conclusão de Mobilidade Acadê...
9,Rematrícula por destrancamento ou descancelamento


In [100]:
d_situacao_academica['sk_d_situacao_academica'] = d_situacao_academica.apply(lambda x: hashlib.sha256(str(x['situacaoMatriculaAtual']).encode()).hexdigest(), axis=1)
d_situacao_academica = d_situacao_academica.reindex(columns=colunas_situacao_academica)
d_situacao_academica.to_csv('data/d_situacao_academica.csv', index=False)
d_situacao_academica

,sk_d_situacao_academica,situacaoMatriculaAtual
0,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,Cancelada por conclusão de curso
1,3ca80f8447d1816b42e6b6ce78876a09ea7ad5e0ae301f...,Cancelada por abandono
2,8362ba253c1a3896ff7da3aaab2e175cb28646495c5e63...,Cancelada por transferência
3,75aeed555b3a43b6e0dddc2c40fd7475c412508eeb27d2...,Cancelada por abandono Definitivo
4,27a82bbf3475a3ffcd91c78ad15adc39fff1f210596c27...,Cancelamento a pedido
5,8336247db87007dd0d6673c56d524eac7df89beccaa6a6...,Cancelamento por ultrapassagem do prazo máximo...
6,924e940f4a6c290b22e8b4a9c1d554a38cfd80b8364c02...,Cancelamento por opção de curso
7,2919d1d61aa9bcc127530e38fc132e937b8b43d21a1938...,Cancelada por rendimento escolar insuficiente
8,5e723f9d214270c85a578cee66bfb97b78dea411c14fa3...,Cancelamento por conclusão de Mobilidade Acadê...
9,e6ec8667d6f4052cd7f8b0691be380ed691e49c3351995...,Rematrícula por destrancamento ou descancelamento


## Fato Desempenho Academico

In [101]:
f_desempenho_academico = df_fato.drop(['periodo', 'periodoIngressoCursoAtual', 'periodosIntegralizados', 'cargaHorariaAcumulada'], axis=1)
f_desempenho_academico.head(1)

,cod_Disciplina,disciplina,grau_disciplina,situacao_disciplina,matriculaDRE,cra,cr,periodoIngressoUFRJ,cursoIngressoUFRJ,cursoAtual,situacaoMatriculaAtual,sexo,nome_completo,ano,semestre
0,MAA123,Algebra para Informatica,060,Aprovado,0,7.6,7.6,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,Cancelada por conclusão de curso,M,Otávio Aragão,2000,1


In [102]:
# D_SITUACAO_ACADEMICA
f_desempenho_academico = pd.merge(f_desempenho_academico, d_situacao_academica, on=['situacaoMatriculaAtual'], how='left')
f_desempenho_academico.drop(['situacaoMatriculaAtual'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,8435,6649,58618,69420,97296
cod_Disciplina,MAB489,MAB471,MAB624,FIT111,ICP245
disciplina,Banco de Dados I,Compiladores I,Números Inteiros Criptografia,Física I,Circuitos Lógicos
grau_disciplina,081,093,100,074,020
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Reprovadomedia
matriculaDRE,198,152,1423,1688,2618
cra,7.4,7.8,8.4,6.8,4.9
cr,7.4,8,8.4,7.2,3.3
periodoIngressoUFRJ,2001/1,2001/1,2011/1,2013/1,2020/2
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Ciência da Computação,Bach em Ciências Matemáticas e da Terra,Bacharelado em Ciência da Computação
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Ciência da Computação,Ciência da Computação


In [103]:
# D_PERIODO
f_desempenho_academico = pd.merge(f_desempenho_academico, d_periodo, on=['ano', 'semestre'], how='left')
f_desempenho_academico.drop(['ano', 'semestre'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,483,83807,7636,65562,50592
cod_Disciplina,IEE115,MAB230,MAB232,FIW125,MAB624
disciplina,Economia,Cálculo Numérico (CC),Programação Linear I,"Mecânica, Oscilação e Ondas",Números Inteiros Criptografia
grau_disciplina,068,067,057,072,047
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Reprovadomedia
matriculaDRE,8,2077,179,1588,1168
cra,7.1,6.7,6,5.1,4.1
cr,6.9,6.9,6.3,7.5,3.8
periodoIngressoUFRJ,2000/1,2016/1,2001/1,2012/1,2009/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Ciência da Computação,Bacharelado em Informática,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Ciência da Computação,Bacharelado em Ciência da Computação


In [104]:
# D_CURSO
f_desempenho_academico = pd.merge(f_desempenho_academico, d_curso, on=['cursoIngressoUFRJ', 'cursoAtual'], how='left')
f_desempenho_academico.drop(['cursoIngressoUFRJ', 'cursoAtual'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,42903,49449,2007,74293,35952
cod_Disciplina,MAD243,MAB232,MAB607,MAB605,FIT121
disciplina,Estatística e Probabilidade,Programação Linear I,Empreendimento em Informática,Recuperação de Informação,Física II
grau_disciplina,068,016,062,089,069
situacao_disciplina,Aprovado,Reprovadomedia,Aprovado,Aprovado,Aprovado
matriculaDRE,983,1137,40,1814,827
cra,9,5,6.2,6.4,6.9
cr,9.4,3.4,6.7,8.4,6.8
periodoIngressoUFRJ,2007/2,2008/2,2000/1,2014/1,2006/1
sexo,F,M,M,M,M
nome_completo,Pedro Miguel Gonçalves,Vinicius Pereira,Srta. Rafaela Pinto,Henrique Duarte,Eduardo Cunha


In [105]:
# D_ALUNO
f_desempenho_academico = pd.merge(f_desempenho_academico, d_aluno, on=['nome_completo', 'sexo', 'matriculaDRE'], how='left')
f_desempenho_academico.drop(['nome_completo', 'sexo', 'matriculaDRE'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,87188,77505,75745,16321,6730
cod_Disciplina,MAB121,MAB471,MAB624,MAB489,MAB606
disciplina,Computação I,Compiladores I,Números Inteiros Criptografia,Banco de Dados I,Tóp Esp em Programação
grau_disciplina,097,074,021,053,093
situacao_disciplina,Aprovado,Aprovado,Reprovadomedia,Aprovado,Aprovado
cra,6.7,6.7,3.1,7,8.4
cr,6.7,8.8,3,7.7,8.3
periodoIngressoUFRJ,2017/1,2015/1,2014/2,2002/2,2001/1
sk_d_situacao_academica,65cb84f03d1e58898e1ea2b11194c5ff6096ad76b600de...,65cb84f03d1e58898e1ea2b11194c5ff6096ad76b600de...,db656cf69bbccbdfda7980c6e86a0c0670f1085cdaa09f...,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...
sk_d_periodo,dffcc977a35e3e941a5567f71d111418e49a25109152d0...,a410ca8badb0e813980534c7b60a5425432d4ff5921a13...,9bbebbaaa8b874af4d5c1a4083cc49bfd3a98a897cd1c0...,c2e8c4d907eb354cddc82e7ae2cfeb2f4fe821c08a8c10...,551f1b6da3d076f00cf0f65b0bbb08ee6e88d26efd4abf...
sk_d_curso,b73a0b12bf115c48af094f97260abc5aeac85cc2e47360...,a80f5dd573274667573b525e0750e7915a3bc967190075...,a80f5dd573274667573b525e0750e7915a3bc967190075...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...


In [106]:
# D_DISCIPLINA
f_desempenho_academico = pd.merge(f_desempenho_academico, d_disciplina, on=['cod_Disciplina', 'disciplina'], how='left')
f_desempenho_academico.drop(['cod_Disciplina', 'disciplina'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,79635,9900,70630,10997,57156
grau_disciplina,076,095,000,071,096
situacao_disciplina,Aprovado,Aprovado,Reprovadomedia,Aprovado,Aprovado
cra,7.9,8.8,3,7.6,6.5
cr,5.6,8.8,1.1,7.2,7.4
periodoIngressoUFRJ,2015/1,2001/2,2013/1,2001/2,2010/2
sk_d_situacao_academica,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,8336247db87007dd0d6673c56d524eac7df89beccaa6a6...,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...
sk_d_periodo,dddd8a22f99ec19d35bd2f6c9714ba305614d47f3ca371...,ef0825aa83d176e856dbe4c1c8338fd0105a89cfe3caf1...,dffcc977a35e3e941a5567f71d111418e49a25109152d0...,c2db217ea7a34191c8f55a2acd4ac37c35611dc1946a7c...,e54a2041a494a6534e33f777b64c7912f8a2cc5a4e9a33...
sk_d_curso,69f98f9a931103c9534d1881425cd98f8d920686439325...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,69f98f9a931103c9534d1881425cd98f8d920686439325...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,69f98f9a931103c9534d1881425cd98f8d920686439325...
sk_d_aluno,59279341ea59fbf34025024596b670b6df2c9f80e71b9a...,6af1f692e9496c6d0b668316eccb93276ae6b6774fa728...,319f6d0f44f72b8278c5abaf74ad0357c9d13f8d732824...,d8d1790737d57ac4fe91a2c0a28087c0a97c81f5dc6b19...,a73531f89e6bcbc2a72a8b08ac33122114e19ffbf89d9d...
sk_d_disciplina,70c5438681f6efe2902a4b9e78fd4488412a85544fdf14...,977c66f9828c8b73e4820585c260e13e2d66dba1ab0f4c...,0ab06a7ea6ce32bf8dcd0db37a5dfd4e4bf57da8a2065f...,def4682d25ac08dff9e665120fd38abc76b1eef99aece1...,c48d5518aa1a2c50460fed03089c688d323629c0d27e52...


In [107]:
f_desempenho_academico.drop(['periodoIngressoUFRJ'], axis=1, inplace=True)

In [108]:
f_desempenho_academico.sample(5).T

,59814,90397,34994,5800,17208
grau_disciplina,094,092,003,070,075
situacao_disciplina,Aprovado,Aprovado,Reprovadomedia,Aprovado,Aprovado
cra,5.2,6.1,3.3,7.9,6
cr,4.3,6.1,2.1,7.5,7
sk_d_situacao_academica,65cb84f03d1e58898e1ea2b11194c5ff6096ad76b600de...,65cb84f03d1e58898e1ea2b11194c5ff6096ad76b600de...,75aeed555b3a43b6e0dddc2c40fd7475c412508eeb27d2...,3ca80f8447d1816b42e6b6ce78876a09ea7ad5e0ae301f...,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...
sk_d_periodo,95f4b096690274bdd3cec9f2a44807f912a00e93adf5d1...,8295bff373057234531627dbad9d33550c13492997845e...,7ed6967ca706ff12a3f16ffe89128eac5f23db35b6e22e...,c291fe829d40dc63fc4fa578f6208f425064945d31ca82...,7ed6967ca706ff12a3f16ffe89128eac5f23db35b6e22e...
sk_d_curso,b73a0b12bf115c48af094f97260abc5aeac85cc2e47360...,acbc30f06ec39601955703a6f4fb05791a031a8f7755d1...,69f98f9a931103c9534d1881425cd98f8d920686439325...,dbde55e27224a8c832119eba0c54cb25a32bf6f6417dd2...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...
sk_d_aluno,146c09db1bc3d53de430b2539d042fc870a84996071db5...,83d7ce01060ca9e9dc54af6c215800d77b11bad522966f...,d1c78c9aa5dcb0991f46b25fbaaa359d7d5823ac7a2a94...,d2f483672c0239f6d7dd3c9ecee6deacbcd59185855625...,a73b320dc0d3a57c03f897eb28ca91e623c5ee635db594...
sk_d_disciplina,e94a1bf5c7c337230e1d228b3cd4b6f5f958b200ca661e...,df6f4b64594be0afd511dcc55550537e306ab96a9c9424...,de4f94b5804ebaccfa8f5d68972cfbd7693a996d28f224...,3a04156996d42b930ed55f57137bf2db8026c209726bed...,e7141983565bdb9eb94062a3c0e4464d8ef6455dcc5d66...


In [109]:
f_desempenho_academico.to_csv('data/f_desempenho_academico.csv', index=False)

# Analise de disciplinas com nomes errados

In [110]:
d_disciplina[d_disciplina['cod_Disciplina'] == 'MAB613']

,sk_d_disciplina,cod_Disciplina,disciplina
51,69c74b0dc4bd99f8f39728f26b5edb757d2f12f54e08ea...,MAB613,Met Desenv de Sist Informacao
118,4831abbc49e3963e443cfe77fde3e1f4966cbaa7adf118...,MAB613,Met Desenv de Sist Informação


In [113]:
d_disciplina[d_disciplina['cod_Disciplina'] == 'MAB601']
# EEL410

,sk_d_disciplina,cod_Disciplina,disciplina
346,32fb0f4baaf5782b9b0b336cab51bcc29f92b887898c15...,EEL410,Eletronica II
1500,c3beef48b7eb309b6210dc537905dd2d313838f9de1093...,EEL410,Eletrônica II


In [112]:
d_disciplina[d_disciplina['cod_Disciplina'].duplicated()]

,sk_d_disciplina,cod_Disciplina,disciplina
61,e4b4dc9868db99e7a7c2f8c49d7597179bcf9697daf319...,MAB601,Top Esp em Eng de Software
118,4831abbc49e3963e443cfe77fde3e1f4966cbaa7adf118...,MAB613,Met Desenv de Sist Informação
358,bb19397c608b8ad6e1e91f353d8aeaa3c0109a13a33ecb...,MAB603,Gestão do Conhecimento
383,58ae4c22500c1bf1a3dd156b6405367f685508d5a856ab...,MAB230,Cálculo Numérico (CC)
417,70c5438681f6efe2902a4b9e78fd4488412a85544fdf14...,MAB602,Dw no Sup à Tomada de Decisão
419,c6fdafe7054dc9166dd654d7d98ef1d8da16718f40133a...,MAB600,Dados Semi-estruturados e XML
460,ad46eeb3845a2b7de8e5ceebc4230a1850be2509cad643...,MAB532,Mineração de Dados
462,a8cff0b3f3cdc6991275d971f5fef3fdfbbf8d6393afbf...,EEH210,Engenharia e Meio Ambiente
471,6350703008dfb26f834d42c1683bf89f50315ddbea870e...,IEE304,Desenvolvimento Socioeconomico
474,310f5c6e86c7e6cbe976fa5706be1805c8d1f8b59bd6ca...,MAB120,Computação I (CC)
